In [ ]:
import os
import cv2
import mediapipe as mp
from rembg import remove
import numpy as np

# Initialize MediaPipe hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Create output directory
os.makedirs('./lined_images', exist_ok=True)

# Input folder
input_folder = './palm'

# Hand landmark detection function
def process_image(image_path):
    # Read and remove background
    with open(image_path, 'rb') as f:
        input_data = f.read()
    output_data = remove(input_data)
    nparr = np.frombuffer(output_data, np.uint8)
    img_no_bg = cv2.imdecode(nparr, cv2.IMREAD_UNCHANGED)

    # If image has alpha, convert to RGB
    if img_no_bg.shape[2] == 4:
        bg_removed = cv2.cvtColor(img_no_bg, cv2.COLOR_BGRA2BGR)
    else:
        bg_removed = img_no_bg

    # Convert for MediaPipe
    img_rgb = cv2.cvtColor(bg_removed, cv2.COLOR_BGR2RGB)

    with mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5) as hands:
        results = hands.process(img_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    bg_removed,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
                )

    return bg_removed

# Process all images in folder
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        input_path = os.path.join(input_folder, filename)
        print(f"Processing: {filename}")
        lined_img = process_image(input_path)

        output_path = os.path.join('./lined_images', f"lined_{filename}")
        cv2.imwrite(output_path, lined_img)

print("✅ All images processed and saved in ./lined_images/")


In [ ]:
import os
import cv2
import numpy as np
from keras.models import load_model
import mediapipe as mp
from rembg import remove

# Load trained model
model = load_model('./palm_model.h5')  # adjust name if different

# Create output folder
os.makedirs('./detected_palms', exist_ok=True)

# MediaPipe for locating hands
mp_hands = mp.solutions.hands

def predict_palm(crop):
    resized = cv2.resize(crop, (224, 224))  # size used in training
    norm = resized.astype('float32') / 255.0
    input_tensor = np.expand_dims(norm, axis=0)
    prediction = model.predict(input_tensor, verbose=0)
    return prediction[0][0] > 0.5  # adjust threshold if needed

def detect_and_draw_box(image_path, output_path):
    # Remove background
    with open(image_path, 'rb') as f:
        input_data = f.read()
    no_bg = remove(input_data)
    nparr = np.frombuffer(no_bg, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_UNCHANGED)

    if img.shape[2] == 4:
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

    h, w, _ = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    found = False
    with mp_hands.Hands(static_image_mode=True) as hands:
        result = hands.process(img_rgb)
        if result.multi_hand_landmarks:
            for hand in result.multi_hand_landmarks:
                indices = [0, 1, 5, 9, 13, 17]
                points = [(int(hand.landmark[i].x * w), int(hand.landmark[i].y * h)) for i in indices]

                x_min = max(min(p[0] for p in points) - 20, 0)
                y_min = max(min(p[1] for p in points) - 20, 0)
                x_max = min(max(p[0] for p in points) + 20, w)
                y_max = min(max(p[1] for p in points) + 20, h)

                cropped = img[y_min:y_max, x_min:x_max]

                if cropped.size == 0:
                    continue

                if predict_palm(cropped):
                    found = True
                    # Draw box
                    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

    cv2.imwrite(output_path, img)
    return found

# Process folder of images
input_folder = './palm'
for file in os.listdir(input_folder):
    if file.lower().endswith(('.png', '.jpg', '.jpeg')):
        in_path = os.path.join(input_folder, file)
        out_path = os.path.join('./detected_palms', f'boxed_{file}')
        success = detect_and_draw_box(in_path, out_path)
        print(f"{'✅' if success else '❌'} Processed: {file}")


: 

In [ ]:
%pip install tensorflow 